# 웹페이지에서 원하는 데이터 추출하고 정리하기

In [1]:
from bs4 import BeautifulSoup 
import requests

headers = { 'User-Agent' : ('Mozilla/5.0 (Windows NT 10.0;Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'), } 
url = 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, "html.parser")
rank = [int(x.get_text()) for x in soup.find_all('div', 'sammyRank')]
len(rank)

50

In [2]:
store_name = list()
main_menu = list()
tmp_url = list()
for x in soup.find_all('div', 'sammyListing'):
    store_name.append(x.find('a').get_text().split("\n")[1])
    main_menu.append(x.find('a').get_text().split("\n")[0])
    tmp_url.append(x.find('a')['href'] if x.find('a')['href'].startswith("https") else "https://www.chicagomag.com" + x.find('a')['href'])

In [3]:
import pandas as pd

In [4]:
data = {'Rank':rank, 'Cafe':store_name, 'Menu':main_menu, 'URL':tmp_url}
df = pd.DataFrame(data)
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


In [5]:
df.to_csv('../data/03. best_sandwiches_list_chicago.csv', sep=',', encoding='UTF-8')

In [6]:
df = pd.read_csv('../data/03. best_sandwiches_list_chicago.csv', index_col=0)
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


In [7]:
price = list()
address = list()

for i in df.index:
    response = requests.get(df["URL"][i], headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    price_n_address = soup.find('p', 'addy').get_text().split()
    price.append(price_n_address[0][:-1])
    address.append(' '.join(price_n_address[1:-2]))

In [8]:
df['Price'] = price
df['Address'] = address

df = df.loc[:, ['Rank', 'Cafe', 'Menu', 'Price', 'Address']]
df.set_index('Rank', inplace=True)
df.head(15)

,Cafe,Menu,Price,Address
Rank,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,"
6,Hendrickx Belgian Bread Crafter,Belgian Chicken Curry Salad,$7.25,100 E. Walton
7,Acadia,Lobster Roll,$16,"1639 S. Wabash Ave.,"
8,Birchwood Kitchen,Smoked Salmon Salad,$10,"2211 W. North Ave.,"
9,Cemitas Puebla,Atomica Cemitas,$9,"3619 W. North Ave.,"


In [9]:
df.to_csv('../data/03. best_sandwiches_list_chicago2.csv', sep=',', encoding='UTF-8')

# 맛집 위치를 지도에 표기하기

In [10]:
import folium
import googlemaps
import numpy as np
from tqdm import notebook

In [11]:
df = pd.read_csv('../data/03. best_sandwiches_list_chicago2.csv', index_col=0)
df.head(5)

,Cafe,Menu,Price,Address
Rank,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,"
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,"
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,"
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,"
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,"


In [12]:
%load_ext dotenv
%dotenv
import os
gmap_key = os.environ.get("GOOGLE_MAP_KEY")
gmaps = googlemaps.Client(key=gmap_key)

In [13]:
lat = []
lng = []

for n in notebook.tqdm(df.index):
    if df['Address'][n] != 'Multiple':
        target_name = df['Address'][n]+', '+'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
        
    else:
        lat.append(np.nan)
        lng.append(np.nan)

  0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
len(lat), len(lng)

(50, 50)

In [15]:
df['lat'] = lat
df['lng'] = lng
df.head(15)

,Cafe,Menu,Price,Address,lat,lng
Rank,,,,,,
1,Old Oak Tap,BLT,$10,"2109 W. Chicago Ave.,",41.895558,-87.679967
2,Au Cheval,Fried Bologna,$9,"800 W. Randolph St.,",41.884639,-87.647590
3,Xoco,Woodland Mushroom,$9.50,"445 N. Clark St.,",41.890523,-87.630783
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston,",42.058322,-87.683748
5,Publican Quality Meats,PB&L,$10,"825 W. Fulton Mkt.,",41.886604,-87.648536
6,Hendrickx Belgian Bread Crafter,Belgian Chicken Curry Salad,$7.25,100 E. Walton,41.900250,-87.625078
7,Acadia,Lobster Roll,$16,"1639 S. Wabash Ave.,",41.859054,-87.625201
8,Birchwood Kitchen,Smoked Salmon Salad,$10,"2211 W. North Ave.,",41.910203,-87.682875
9,Cemitas Puebla,Atomica Cemitas,$9,"3619 W. North Ave.,",41.909756,-87.717673


In [16]:
mapping = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=11)

for n in df.index:
    if df['Address'][n] != 'Multiple':
        folium.Marker([df['lat'][n], df['lng'][n]], 
                                      popup=folium.Popup(f"<b>{df['Cafe'][n]}</b><br>{df['Menu'][n]} - {df['Price'][n]}", min_width=200, max_width=200)
                                      ).add_to(mapping)

mapping